# Toy Example of SBI in Population Genetics

In population genetics, forward or coalescent simulations can produce synthetic data under different demographic scenarios.

In this notebook we apply simulation‑based inference to infer the effective population size N_e of a constant‑size population from the number of segregating sites observed in simulated DNA sequences. 

Although real applications would use more sophisticated summary statistics, this toy example demonstrates the whole workflow.

## Simulation model

We assume a single population evolving under the neutral coalescent. Chromosomes are simulated with length 1 Mb, a fixed mutation rate and recombination rate. The simulator function will take \(N_e\) as input and return the number of segregating sites as a simple summary statistic.

In [55]:
import torch
from sbi.inference import NPE
from sbi.utils import BoxUniform
import msprime

In [36]:
# Simulator wrapper for sbi: takes a tensor of shape (num_simulations, 1)
def simulator(theta):
    s_stats = []
    for ne in theta.squeeze().tolist():
        ts = simulate_constant_population(sequence_length=1_000_000, ne=ne, samples=10)
        s = ts.allele_frequency_spectrum()
        s_stats.append([s])
    return torch.tensor(s_stats, dtype=torch.float32)

def simulate_constant_population(sequence_length, ne, samples,
                                 mutation_rate=1e-8,
                                 recombination_rate=1e-8):
    # Define demography
    demography = msprime.Demography()
    demography.add_population(initial_size=ne)

    # Simulate ancestry and mutations
    ts_anc = msprime.sim_ancestry(samples=samples,
                                recombination_rate=recombination_rate,
                                sequence_length=sequence_length,
                                demography=demography)
    ts = msprime.sim_mutations(ts_anc, rate=mutation_rate)

    return ts


In [99]:
# Prior distribution for N_e (log‑uniform between 1e3 and 1e6)
prior = BoxUniform(low=torch.tensor([1e2]), high=torch.tensor([1e4]))

# Generate training data
num_simulations = 100
theta = prior.sample((num_simulations,))

In [100]:
x = simulator(theta)

In [101]:
# Train NPE
inference = NPE(prior=prior)
inference = inference.append_simulations(theta, x.squeeze())
density_estimator = inference.train()
posterior = inference.build_posterior(density_estimator)

/Users/yuxinning/miniconda3/envs/sbi-workshop/lib/python3.12/site-packages/sbi/neural_nets/net_builders/flow.py:149: UserWarning: In one-dimensional output space, this flow is limited to Gaussians
  x_numel = get_numel(


 Neural network successfully converged after 81 epochs.

In [102]:
# Suppose we observed 150 segregating sites
test_theta = prior.sample((2,))
x_o = simulator(test_theta).squeeze()[0]
samples = posterior.sample((2000,), x=x_o)

# Print posterior mean and 90% credible interval
mean_est = samples.mean().item()
ci_low = samples.quantile(0.05).item()
ci_high = samples.quantile(0.95).item()
print('Posterior mean N_e:', mean_est)
print('90% credible interval: [', ci_low, ',', ci_high, ']')

  0%|          | 0/2000 [00:00<?, ?it/s]

Posterior mean N_e: 9014.3701171875
90% credible interval: [ 7992.34130859375 , 9850.2509765625 ]


In [103]:
test_theta

tensor([[8906.6768],
        [7356.4160]])

In [105]:
samples.mean()

tensor(9014.3701)